## First day: list comprehensions and generators

> List comprehensions and generators are in my top 5 favorite Python features leading to clean, robust and Pythonic code. 

In [1]:
from collections import Counter
import calendar
import itertools
import random
import re
import string

import requests

### List comprehensions

Let's dive straight into a practical example. We all know how to use the classic for loop in Python, say I want to loop through a bunch of names title-casing each one:

In [2]:
names = 'pybites mike bob julian tim sara guido'.split()
names

['pybites', 'mike', 'bob', 'julian', 'tim', 'sara', 'guido']

In [3]:
for name in names:
    print(name.title())

Pybites
Mike
Bob
Julian
Tim
Sara
Guido


Then I want to only keep the names that start with A-M, the `strings` module makes it easier (we love Python's standard library!):

In [4]:
first_half_alphabet = list(string.ascii_lowercase)[:13]
first_half_alphabet

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']

In [5]:
new_names = []
for name in names:
    if name[0] in first_half_alphabet:
        new_names.append(name.title())
new_names

['Mike', 'Bob', 'Julian', 'Guido']


Feels verbose, not? 

If you don't know about list comprehensions you might start using them everywhere after seeing the next refactoring:

In [6]:
new_names2 = [name.title() for name in names if name[0] in first_half_alphabet]
new_names2

['Mike', 'Bob', 'Julian', 'Guido']

In [7]:
assert new_names == new_names2

From 4 to 1 lines of code, and it reads pretty well too. That's why we love and stick with Python! 

Here is another example I used recently to do a most common word count on Harry Potter. I used some list comprehensions to clean up the words before counting them:

In [34]:
resp = requests.get('https://bites-data.s3.us-east-2.amazonaws.com/harry.txt')
# print(resp.text)
words = resp.text.lower().split()
words[:5]

['the', 'boy', 'who', 'lived', 'mr.']

In [35]:
cnt = Counter(words)
cnt.most_common(5)

[('the', 202), ('he', 136), ('a', 108), ('and', 100), ('to', 93)]

Hmm should not count stopwords, also:

In [36]:
'-' in words

True

Let's first clean up any non-alphabetic characters:

In [37]:
words = [re.sub(r'\W+', r'', word) for word in words]

In [38]:
'-' in words

False

In [39]:
'the' in words

True

Ok let's filter those stopwords out plus the empty strings caussed by the previous list comprehension:

In [41]:
resp = requests.get('https://bites-data.s3.us-east-2.amazonaws.com/stopwords.txt')
print(resp.text)
stopwords = resp.text.lower().split()
stopwords[:5]  

a
about
above
across
after
afterwards
again
against
all
almost
alone
along
already
also
although
always
am
among
amongst
amoungst
amount
an
and
another
any
anyhow
anyone
anything
anyway
anywhere
are
around
as
at
back
be
became
because
become
becomes
becoming
been
before
beforehand
behind
being
below
beside
besides
between
beyond
bill
both
bottom
but
by
call
can
cannot
cant
co
computer
con
could
couldnt
cry
de
describe
detail
do
done
down
due
during
each
eg
eight
either
eleven
else
elsewhere
empty
enough
etc
even
ever
every
everyone
everything
everywhere
except
few
fifteen
fify
fill
find
fire
first
five
for
former
formerly
forty
found
four
from
front
full
further
get
give
go
had
has
hasnt
have
he
hence
her
here
hereafter
hereby
herein
hereupon
hers
herse"
him
himse"
his
how
however
hundred
i
ie
if
in
inc
indeed
interest
into
is
it
its
itse"
keep
last
latter
latterly
least
less
ltd
made
many
may
me
meanwhile
might
mill
mine
more
moreover
most
mostly
move
much
must
my
myse"
name
namely
ne

['a', 'about', 'above', 'across', 'after']

In [43]:
words[:]
words = [word for word in words if word.strip() and word not in stopwords]
words[:5]

['boy', 'lived', 'mr', 'mrs', 'dursley']

In [44]:
'the' in words

False

Now it looks way better:

In [45]:
cnt = Counter(words)
cnt.most_common(5)

[('dursley', 45),
 ('dumbledore', 35),
 ('said', 32),
 ('mr', 30),
 ('professor', 30)]

What's interesting here is that the first bit of the list comprehension can be an expression like `re.sub`. The final bit can be a compound statement: here we checked for a non-empty word (' ' -> `strip()` -> '' = `False` in Python) `and` we checked `word not in stopwords`. 

Again, a lot is going on in one line of code, but the beauty of it is that it is totally fine, because it reads like plain English :)

### Generators

A generator is a function that returns an iterator. It generates values using the `yield` keyword, when called with next() (a for loop does this implicitly), and it raises a `StopIteration` exception when there are no more values to generate. Let's see what this means with a very simple example:

In [14]:
def num_gen():
    for i in range(5):
        yield i
        
gen = num_gen()

In [16]:
next(gen)

1

In [17]:
# note it takes off where we left it last statement
for i in gen:
    print(i)

2
3
4


In [18]:
# no more values to generate
next(gen)

StopIteration: 

In [19]:
# for catches the exception for us
for i in gen:
    print(i)

> The `StopIteration` error appears because there are no more yield statements in the function. Calling next on the generator after this does not cause it to loop over and start again. - [Generators are Awesome, Learning by Example
](https://pybit.es/generators.html)




Since learning about generators, a common pattern I use is to build up my sequences:

In [20]:
options = 'red yellow blue white black green purple'.split()
options

['red', 'yellow', 'blue', 'white', 'black', 'green', 'purple']

My older code:

In [21]:
def create_select_options(options=options):
    select_list = []
    
    for option in options:
        select_list.append(f'<option value={option}>{option.title()}</option>')
    
    return select_list

In [22]:
from pprint import pprint as pp
pp(create_select_options())

['<option value=red>Red</option>',
 '<option value=yellow>Yellow</option>',
 '<option value=blue>Blue</option>',
 '<option value=white>White</option>',
 '<option value=black>Black</option>',
 '<option value=green>Green</option>',
 '<option value=purple>Purple</option>']


Using a generator you can write this in 2 lines of code - my newer code:

In [23]:
def create_select_options_gen(options=options):    
    for option in options:
        yield f'<option value={option}>{option.title()}</option>'

In [24]:
print(create_select_options_gen())

<generator object create_select_options_gen at 0x10fc8f5a0>


Note that generators are _lazy_ so you need to explicitly consume them by iterating over them, for example by looping over them. Another way is to pass them into the `list()` constructor:

In [28]:
list(create_select_options_gen())

['<option value=red>Red</option>',
 '<option value=yellow>Yellow</option>',
 '<option value=blue>Blue</option>',
 '<option value=white>White</option>',
 '<option value=black>Black</option>',
 '<option value=green>Green</option>',
 '<option value=purple>Purple</option>']

Specially when working with large data sets you definitely want to use generators. Lists can only get as big as they fit memory size. Generators are lazily evaluated meaning that they only hold a certain amount of data in memory at once. Just for the sake of giving Python something to do, let's calculate leap years for a million years, and compare performance of list vs generator:

In [25]:
# list
def leap_years_lst(n=1000000):
    leap_years = []
    for year in range(1, n+1):
        if calendar.isleap(year):
            leap_years.append(year)
    return leap_years

# generator
def leap_years_gen(n=1000000):
    for year in range(1, n+1):
        if calendar.isleap(year):
            yield year

PRO  tip: [since Python 3.3](https://docs.python.org/3/whatsnew/3.3.html) you can use the `yield from` syntax.

In [26]:
# this had me waiting for a few seconds
%timeit -n1 leap_years_lst()

591 ms ± 273 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
# this was instant
%timeit -n1 leap_years_gen()

The slowest run took 5.22 times longer than the fastest. This could mean that an intermediate result is being cached.
911 ns ± 628 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


That is pretty impressive. This is an important concept to know about because Big Data is here to stay!

## Second day: practice

Look at your code and see if you can refactor it to use list comprehensions. Same for generators. Are you building up a list somewhere where you could potentially use a generator?

And/or exercise here, take this list of names:

In [47]:
NAMES = ['arnold schwarzenegger', 'alec baldwin', 'bob belderbos',
         'julian sequeira', 'sandra bullock', 'keanu reeves',
         'julbob pybites', 'bob belderbos', 'julian sequeira',
         'al pacino', 'brad pitt', 'matt damon', 'brad pitt']

Can you write a simple list comprehension to convert these names to title case (brad pitt -> Brad Pitt). Or reverse the first and last name? 

In [55]:
from pprint import pprint
import re

pprint([name.title() for name in NAMES])
pprint([re.sub(r"(\w+)\s(\w+)", r"\2 \1", name).title() for name in NAMES])

['Arnold Schwarzenegger',
 'Alec Baldwin',
 'Bob Belderbos',
 'Julian Sequeira',
 'Sandra Bullock',
 'Keanu Reeves',
 'Julbob Pybites',
 'Bob Belderbos',
 'Julian Sequeira',
 'Al Pacino',
 'Brad Pitt',
 'Matt Damon',
 'Brad Pitt']
['Schwarzenegger Arnold',
 'Baldwin Alec',
 'Belderbos Bob',
 'Sequeira Julian',
 'Bullock Sandra',
 'Reeves Keanu',
 'Pybites Julbob',
 'Belderbos Bob',
 'Sequeira Julian',
 'Pacino Al',
 'Pitt Brad',
 'Damon Matt',
 'Pitt Brad']


Then use this same list and make a little generator, for example to randomly return a pair of names, try to make this work:

    pairs = gen_pairs()
    for _ in range(10):
        next(pairs)

Should print (values might change as random):

    Arnold teams up with Brad
    Alec teams up with Julian

Have fun!

In [63]:
import random

for name in NAMES:
    print(f"{name.title().split()[0]} teams up with {NAMES[random.randint(0, len(names) - 1)].title().split()[0]}")

Arnold teams up with Sandra
Alec teams up with Arnold
Bob teams up with Arnold
Julian teams up with Sandra
Sandra teams up with Alec
Keanu teams up with Sandra
Julbob teams up with Keanu
Bob teams up with Alec
Julian teams up with Keanu
Al teams up with Julbob
Brad teams up with Julbob
Matt teams up with Alec
Brad teams up with Sandra


## Third day: solution / simulate unix pipelines

I hope yesterday's exercise was reasonably doable for you. Here are the answers in case you got stuck:

In [33]:
# list comprehension to title case names
[name.title() for name in NAMES]

['Arnold Schwarzenegger',
 'Alec Baldwin',
 'Bob Belderbos',
 'Julian Sequeira',
 'Sandra Bullock',
 'Keanu Reeves',
 'Julbob Pybites',
 'Bob Belderbos',
 'Julian Sequeira',
 'Al Pacino',
 'Brad Pitt',
 'Matt Damon',
 'Brad Pitt']

In [34]:
# list comprehension to reverse first and last names
# using a helper here to show you that list comprehensions can be passed in functions!

def reverse_first_last_names(name):
    first, last = name.split()
    # ' '.join([last, first]) -- wait we have f-strings now (>= 3.6)
    return f'{last} {first}'

[reverse_first_last_names(name) for name in NAMES]

['schwarzenegger arnold',
 'baldwin alec',
 'belderbos bob',
 'sequeira julian',
 'bullock sandra',
 'reeves keanu',
 'pybites julbob',
 'belderbos bob',
 'sequeira julian',
 'pacino al',
 'pitt brad',
 'damon matt',
 'pitt brad']

In [35]:
def gen_pairs():
    # again a list comprehension is great here to get the first names
    # and title case them in just 1 line of code (this comment took 2)
    first_names = [name.split()[0].title() for name in NAMES]
    while True:
        
        # added this when I saw Julian teaming up with Julian (always test your code!)
        first, second = None, None
        while first == second: 
            first, second = random.sample(first_names, 2)
        
        yield f'{first} teams up with {second}'

In [36]:
pairs = gen_pairs()
for _ in range(10):
    print(next(pairs))

Alec teams up with Julbob
Keanu teams up with Bob
Keanu teams up with Julbob
Julian teams up with Arnold
Bob teams up with Alec
Matt teams up with Alec
Julbob teams up with Brad
Julian teams up with Alec
Julian teams up with Julbob
Julbob teams up with Julian


Another way to get a slice of a generator is using `itertools.islice`:

In [37]:
first_ten = itertools.islice(pairs, 10)
first_ten

In [38]:
list(first_ten)

['Sandra teams up with Julian',
 'Matt teams up with Julian',
 'Al teams up with Julian',
 'Brad teams up with Julian',
 'Alec teams up with Arnold',
 'Matt teams up with Bob',
 'Matt teams up with Julian',
 'Julbob teams up with Julian',
 'Brad teams up with Julian',
 'Julian teams up with Julbob']

### Further practice

Read up on set and dict comprehensions, then try these two Bites:
- [Bite 5. Parse a list of names](https://codechalleng.es/bites/5/) (use a set comprehension in first function)
- [Bite 26. Dictionary comprehensions are awesome](https://codechalleng.es/bites/promo/awesome-dict-comprehensions)

Here is a more advanced generators exercise you can try: [Code Challenge 11 - Generators for Fun and Profit](https://codechalleng.es/challenges/11/)

### Time to share what you've accomplished!

Be sure to share your last couple of days work on Twitter or Facebook. Use the hashtag **#100DaysOfCode**. 

Here are [some examples](https://twitter.com/search?q=%23100DaysOfCode) to inspire you. Consider including [@talkpython](https://twitter.com/talkpython) and [@pybites](https://twitter.com/pybites) in your tweets.

*See a mistake in these instructions? Please [submit a new issue](https://github.com/talkpython/100daysofcode-with-python-course/issues) or fix it and [submit a PR](https://github.com/talkpython/100daysofcode-with-python-course/pulls).*